In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('ElectricCarData_Clean.csv')
data.head(4)

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040


In [4]:
data.columns,data.shape

(Index(['Brand', 'Model', 'AccelSec', 'TopSpeed_KmH', 'Range_Km',
        'Efficiency_WhKm', 'FastCharge_KmH', 'RapidCharge', 'PowerTrain',
        'PlugType', 'BodyStyle', 'Segment', 'Seats', 'PriceEuro'],
       dtype='object'),
 (103, 14))

In [4]:
col_list = list(data.columns)
topview_data = pd.DataFrame()
for col in col_list:
    datum = pd.DataFrame({'colname':col,
                          '#uniq_':[len(data[col].unique())],
                          'dtype':[data[col].dtype],
                          '#missing_':[data[col].isna().sum()],
                          'is_PKey':[len(data[col].unique()) == data.shape[0]]})
    topview_data = pd.concat([topview_data,datum],axis = 0)
    print(f'processing col :{col}')
    
topview_data

processing col :Brand
processing col :Model
processing col :AccelSec
processing col :TopSpeed_KmH
processing col :Range_Km
processing col :Efficiency_WhKm
processing col :FastCharge_KmH
processing col :RapidCharge
processing col :PowerTrain
processing col :PlugType
processing col :BodyStyle
processing col :Segment
processing col :Seats
processing col :PriceEuro


,colname,#uniq_,dtype,#missing_,is_PKey
0,Brand,33,object,0,False
0,Model,102,object,0,False
0,AccelSec,55,float64,0,False
0,TopSpeed_KmH,25,int64,0,False
0,Range_Km,50,int64,0,False
0,Efficiency_WhKm,54,int64,0,False
0,FastCharge_KmH,51,object,0,False
0,RapidCharge,2,object,0,False
0,PowerTrain,3,object,0,False
0,PlugType,4,object,0,False


[Inference:]
> From the above Data Structure Analysis, we can directly infer that on feature individually is the primary key of the data.
> Hence based on the context of the data we must investigate if Brand + Model together is the primary key of the data. 
> Also, we can see that the above data doesnot have any missing values, in order to truly validate we must manually check in the data as sometimes missing values can be replaced by special symbols

In [5]:
data['brand_model_key'] = data['Brand'].astype(str) + data['Model'].astype(str)
len(data['brand_model_key'].unique()), len(data['brand_model_key'].unique()) == data.shape[0]

(102, False)

In [6]:
BM_array = data[['Brand','Model']].drop_duplicates()
BM_array.shape,data[['Brand','Model']].shape

((102, 2), (103, 2))

[Inference:]
> This clearly tells us that there must be a brand & model that is repeating in the columns 'Brand' & 'Model'.
> If this is the case then we must find out whether this brand & model has the same entries of the car-parameters.  

[Missing Values : FastCharge_KmH]
> Based on the nature of missing values we can use direct imputation method

In [7]:
data.loc[data['FastCharge_KmH'] == '-',['Brand','Model','FastCharge_KmH']]

,Brand,Model,FastCharge_KmH
57,Renault,Twingo ZE,-
68,Renault,Kangoo Maxi ZE 33,-
77,Smart,EQ forfour,-
82,Smart,EQ fortwo coupe,-
91,Smart,EQ fortwo cabrio,-


In [10]:
brand_modellist = list(data.loc[data['FastCharge_KmH'] == '-'].brand_model_key.unique())
BM_dict = {}
for BM in brand_modellist:
    BM_dict[BM] = []

In [11]:
BM_dict

{'Renault Twingo ZE': [],
 'Renault Kangoo Maxi ZE 33': [],
 'Smart EQ forfour ': [],
 'Smart EQ fortwo coupe': [],
 'Smart EQ fortwo cabrio': []}

[EDA]

[Feature : AccelSec] > Accleration Seconds

In [23]:
data.AccelSec.describe()

(2.1,
 count    103.000000
 mean       7.396117
 std        3.017430
 min        2.100000
 25%        5.100000
 50%        7.300000
 75%        9.000000
 max       22.400000
 Name: AccelSec, dtype: float64)

In [21]:
data.groupby('Brand').agg(avg_AccelSec = pd.NamedAgg('AccelSec','mean'),
                          min_AccelSec = pd.NamedAgg('AccelSec','min'),
                          max_AccelSec = pd.NamedAgg('AccelSec','max')).reset_index().sort_values('avg_AccelSec',ascending = True)

,Brand,avg_AccelSec,min_AccelSec,max_AccelSec
15,Lucid,2.800000,2.8,2.8
24,Porsche,3.500000,2.8,4.0
30,Tesla,4.092308,2.1,7.0
23,Polestar,4.700000,4.7,4.7
11,Jaguar,4.800000,4.8,4.8
32,Volvo,4.900000,4.9,4.9
1,Audi,5.566667,3.5,6.8
2,BMW,6.250000,4.0,7.3
8,Ford,6.400000,6.0,7.0
4,CUPRA,6.500000,6.5,6.5


[Feature : TopSpeed_KmH]: Top Speed in kmph

In [24]:
data.TopSpeed_KmH.describe()

count    103.000000
mean     179.194175
std       43.573030
min      123.000000
25%      150.000000
50%      160.000000
75%      200.000000
max      410.000000
Name: TopSpeed_KmH, dtype: float64

In [26]:
data.groupby('Brand').agg(avg_TopSpeed_KmH = pd.NamedAgg('TopSpeed_KmH','mean'),
                          min_TopSpeed_KmH = pd.NamedAgg('TopSpeed_KmH','min'),
                          max_TopSpeed_KmH = pd.NamedAgg('TopSpeed_KmH','max')).reset_index().sort_values('avg_TopSpeed_KmH',ascending = False)

,Brand,avg_TopSpeed_KmH,min_TopSpeed_KmH,max_TopSpeed_KmH
24,Porsche,254.000000,250,260
15,Lucid,250.000000,250,250
30,Tesla,244.461538,180,410
23,Polestar,210.000000,210,210
1,Audi,200.000000,180,240
11,Jaguar,200.000000,200,200
3,Byton,190.000000,190,190
32,Volvo,180.000000,180,180
8,Ford,180.000000,180,180
18,Mercedes,173.333333,140,200


[Feature: Range_Km] > Mileage

In [27]:
data.Range_Km.describe()

count    103.000000
mean     338.786408
std      126.014444
min       95.000000
25%      250.000000
50%      340.000000
75%      400.000000
max      970.000000
Name: Range_Km, dtype: float64

In [28]:
data.groupby('Brand').agg(avg_Range_Km = pd.NamedAgg('Range_Km','mean'),
                          min_Range_Km = pd.NamedAgg('Range_Km','min'),
                          max_Range_Km = pd.NamedAgg('Range_Km','max')).reset_index().sort_values('avg_Range_Km',ascending = False)

,Brand,avg_Range_Km,min_Range_Km,max_Range_Km
15,Lucid,610.000000,610,610
14,Lightyear,575.000000,575,575
30,Tesla,500.769231,310,970
4,CUPRA,425.000000,425,425
23,Polestar,400.000000,400,400
8,Ford,395.000000,340,450
24,Porsche,388.000000,365,425
32,Volvo,375.000000,375,375
3,Byton,371.666667,325,400
11,Jaguar,365.000000,365,365
